In [ ]:
# Add outstanding balance (overdue loan balance + any outsanding balance) to the member info
#
# This is spread between "Loans" > "Overdue Only" in MyTurn, and the "Outsatnding Balances"
# report in MyTurn.  Combining it one place helps identify people with heavy outsanding balances.
import pandas as pd

from chtl_helpers import balance_to_float

users = pd.read_pickle('output/members.pkl')
balances = pd.read_csv(
    'input/outstanding-balances.csv',
    # Skip duplicative personal info, just ID and balance is enough.
    usecols=('ID', 'Amount'),
    dtype={
        # Make the 'Member ID' a string for compatibility with other tables. Occassionally set it to something other than a number.
        'ID': 'string',
    },
    # Convert "$0.00" strings to actual numbers
    converters={'Amount': balance_to_float},
).rename(columns={'ID': 'Membership ID'}).set_index('Membership ID')

loans = pd.read_pickle('output/loans.pkl')
late_fees = loans.groupby('Membership ID').sum()

# De-anonymized user data so we can pull emails.
# WARNING: The input data should NOT be committed
raw_deanonymized_members = pd.read_csv('input_with_personal_info/users-snapshot.csv', usecols = ['Email', 'Member ID'], dtype = {'Member ID': 'object'}).rename(
    {'Member ID': 'Membership ID'}, axis=1)

users['Late Fees'] = users.join(late_fees['Late Fees To Date'], on='Membership ID')['Late Fees To Date'].fillna(0.0).round(2)
users['Balance'] = users.join(balances['Amount'], on='Membership ID')['Amount'].fillna(0.0).round(2)
users['Balance Including Late Fees'] = (users['Late Fees'] + users['Balance']).round(2)

high_late_fees = users[users['Balance Including Late Fees'] > 20][['Balance Including Late Fees', 'Balance', 'Late Fees', 'Membership ID']]
high_late_fees_with_email = high_late_fees.join(raw_deanonymized_members.set_index('Membership ID'), on='Membership ID').sort_valuesthe two objects.

    left: use calling frame’s index (or column if on is specified)

    right: use other’s index.

    outer: form union of calling frame’s('Balance Including Late Fees', ascending=False)
high_late_fees_with_email.to_csv('output_with_personal_info/high_late_fees.csv')


high_late_fees.plot.hist()
